In [1]:
import os

In [2]:
print(f"current dir")
curr_dir = %pwd
print(f"{curr_dir}")

print('\n')
print(f"project dir")
os.chdir("../")
%pwd

current dir
d:\college_work\4th year\Sem7th\Project\MLOPS\MLOPS-Wine-Quality-Prediction\research


project dir


'd:\\college_work\\4th year\\Sem7th\\Project\\MLOPS\\MLOPS-Wine-Quality-Prediction'

Entity management

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

Adding configuration manager update

In [4]:
from mlproject.constants import  *
from mlproject.utils.common import read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])
        
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        
        return data_ingestion_config

Updating the componenets

In [6]:
import requests
import urllib.request as request
import zipfile
from mlproject import logger
from mlproject.utils.common import get_size

In [7]:
class DataIngestion:
    def __init__(self, config:DataIngestionConfig):
        self.config  = config
        
        
    def download_file(self):
        """
        Download the zip file
        """
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename}, download! with the following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")
            
    def extract_zip_file(self):
        """
        Extract the download zip file 
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            

Creating the pipline for data ingestion

In [9]:
try: 
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-12-09 20:31:19,456: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-12-09 20:31:19,459: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-09 20:31:19,461: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-12-09 20:31:19,464: INFO: common: created directory at: artifacts]
[2023-12-09 20:31:19,467: INFO: common: created directory at: artifacts/data_ingestion]
[2023-12-09 20:31:24,811: INFO: 2881582567: artifacts/data_ingestion/data.zip, download! with the following info: 
Server: nginx
Date: Sat, 09 Dec 2023 15:02:54 GMT
Content-Type: application/zip
Content-Length: 100254
Last-Modified: Sat, 09 Dec 2023 15:02:28 GMT
Connection: close
ETag: "65748184-1879e"
Strict-Transport-Security: max-age=63072000; includeSubDomains; preload
X-Frame-Options: sameorigin
X-Content-Type-Options: nosniff
X-XSS-Protection: 1; mode=block
Content-Security-Policy: default-src 'none'; media-src 'self'; style-src 'none' 'unsafe-inline'; img-src 'self' 